In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import  matplotlib.pyplot as plt 
from pathlib import Path

# sns.color_palette("colorblind")
# sns.set_palette(sns.color_palette("colorblind"))
sns.set_palette(sns.color_palette("muted"))

pd.set_option('display.max_columns', None)

In [12]:
city_w = pd.read_csv('./datasets/hw_8/city_weather.csv')
district_to_city = pd.read_csv('./datasets/hw_8/district_to_city.csv')

In [48]:
city_w.describe()

,day_temperatre,night_temperature
count,2166.000000,2166.000000
mean,5.107664,-4.294645
std,6.674888,7.335564
min,-25.000000,-35.000000
25%,1.025000,-7.300000
50%,4.500000,-3.000000
75%,9.100000,0.900000
max,26.500000,11.900000


In [80]:
city_district = dict()

In [82]:
for k,v in district_to_city.set_index('city').to_dict()['federal_district'].items():
    k,v = k.lower(), v.lower()
    if v in city_district.keys():
        city_district[v].append(k)
    else:
        city_district[v] = []

In [84]:
city_district

{'центральный  фо': ['брянск',
  'владимир',
  'воронеж',
  'иваново',
  'калуга',
  'кострома',
  'курск',
  'липецк',
  'москва',
  'орел',
  'рязань',
  'смоленск',
  'тамбов',
  'тверь',
  'тула',
  'ярославль'],
 'северо-западный фо': ['вологда',
  'калининград',
  'петрозаводск',
  'сыктывкар',
  'санкт-петербург',
  'мурманск',
  'великий новгород',
  'псков'],
 'уральский фо': ['курган',
  'екатеринбург',
  'тюмень',
  'ханты-мансийск',
  'челябинск'],
 'приволжский фо': ['киров',
  'йошкар-ола',
  'саранск',
  'нижний новгород',
  'оренбург',
  'пенза',
  'пермь',
  'самара',
  'саратов',
  'казань',
  'ижевск',
  'ульяновск',
  'чебоксары'],
 'сибирский фо': ['барнаул',
  'улан-удэ',
  'чита',
  'иркутск',
  'кемерово',
  'красноярск',
  'новосибирск',
  'омск',
  'томск',
  'кызыл',
  'абакан'],
 'дальневосточный фо': ['биробиджан',
  'петропавловск-камчатский',
  'магадан',
  'владивосток',
  'якутск',
  'южно-сахалинск',
  'хабаровск',
  'анадырь'],
 'южный фо': ['астрахан

In [86]:
cities_list = list()
for v in city_district.values():
    cities_list.extend(v)

### С помощью подхода MapReduce посчитайте  
#### 1.  среднюю погоду по каждому городу за представленный период;

In [169]:
# c_temp = {v:[] for v in cities_list}
c_temp = dict()

#### Mapper 
я не стал записывать в csv - по сути словарь с массивами те же агрегации, делить csv на подтаски и запускать батчами?   
заморочусь в следующий раз) у меня дедлайн) 

In [172]:
with open('./datasets/hw_8/city_weather.csv', encoding='utf-8') as cfile:
    for i,line in enumerate(cfile):
        if i == 0:
            continue
        city, _, day_t, night_t = map(str.lower, line.strip().split(','))
        if city in c_temp.keys():
            c_temp[city].append((float(day_t) + float(night_t))/2)
        else:
            c_temp[city] = [(float(day_t) + float(night_t))/2]

#### Shuffle -  пропускаем, у нас нету нод
#### Reducer

In [183]:
# c_temp_reduced = {v:{'cnt': 0, 'temp':0} for v in cities_list}
c_temp_reduced = dict()

In [185]:
for city, temp_list in c_temp.items():
    for temp in temp_list:
        if city in c_temp_reduced.keys():
            c_temp_reduced[city]['cnt'] += 1
            c_temp_reduced[city]['temp'] += temp
        else:
            c_temp_reduced[city] = {'cnt': 1, 'temp': temp}

In [243]:
c_temp_reduced_final = dict()

In [245]:
for city, payload in c_temp_reduced.items():
    c_temp_reduced_final[city] = round(payload['temp']/ payload['cnt'], 2)
c_temp_reduced_final

{'оренбург': 0.42,
 'липецк': 4.3,
 'челябинск': 0.84,
 'томск': -3.71,
 'ханты-мансийск': -2.99,
 'курган': 0.3,
 'казань': 1.97,
 'киров': 0.14,
 'тула': 3.7,
 'ижевск': 0.48,
 'ставрополь': 6.91,
 'брянск': 4.58,
 'кемерово': -3.92,
 'псков': 2.5,
 'петрозаводск': -0.13,
 'йошкар-ола': 2.26,
 'улан-удэ': -5.01,
 'иркутск': -3.39,
 'калининград': 4.44,
 'мурманск': -5.76,
 'иваново': 1.94,
 'омск': -2.49,
 'майкоп': 9.46,
 'волгоград': 5.5,
 'архангельск': -2.75,
 'нальчик': 6.95,
 'салехард': -8.35,
 'санкт-петербург': 1.66,
 'чебоксары': 2.05,
 'орел': 4.44,
 'элиста': 7.2,
 'уфа': -0.11,
 'благовещенск': -4.45,
 'сыктывкар': -1.92,
 'краснодар': 9.1,
 'ростов': 2.09,
 'самара': 2.36,
 'биробиджан': -2.78,
 'вологда': 0.22,
 'анадырь': -12.63,
 'владимир': 2.5,
 'грозный': 7.71,
 'петропавловск-камчатский': -2.34,
 'нижний новгород': 2.55,
 'якутск': -17.18,
 'кызыл': -8.55,
 'кострома': 1.6,
 'тамбов': 4.42,
 'саранск': 2.89,
 'екатеринбург': -0.12,
 'рязань': 3.99,
 'хабаровск': 

### С помощью подхода MapReduce посчитайте  
#### 2. среднюю температуру по федеральному округу;

все те же шаги что и в 1ом + поитерируемся по словарю <b> city_district</b> для определения фед.округа города

In [289]:
city_district_temp = {k:{'cnt':0, 'temp':0} for k in city_district.keys()}
city_district_temp

{'центральный  фо': {'cnt': 0, 'temp': 0},
 'северо-западный фо': {'cnt': 0, 'temp': 0},
 'уральский фо': {'cnt': 0, 'temp': 0},
 'приволжский фо': {'cnt': 0, 'temp': 0},
 'сибирский фо': {'cnt': 0, 'temp': 0},
 'дальневосточный фо': {'cnt': 0, 'temp': 0},
 'южный фо': {'cnt': 0, 'temp': 0},
 'северо-кавказский фо': {'cnt': 0, 'temp': 0}}

In [291]:
for city, temp in c_temp_reduced_final.items():
    for district, cities in city_district.items():
        if city in cities:
            city_district_temp[district]['cnt'] += 1
            city_district_temp[district]['temp'] += temp

In [293]:
city_district_temp_final = dict()
city_district_temp

{'центральный  фо': {'cnt': 15, 'temp': 55.43},
 'северо-западный фо': {'cnt': 8, 'temp': 2.960000000000001},
 'уральский фо': {'cnt': 5, 'temp': -1.7500000000000002},
 'приволжский фо': {'cnt': 11, 'temp': 18.42},
 'сибирский фо': {'cnt': 10, 'temp': -42.660000000000004},
 'дальневосточный фо': {'cnt': 8, 'temp': -52.91},
 'южный фо': {'cnt': 5, 'temp': 31.709999999999997},
 'северо-кавказский фо': {'cnt': 4, 'temp': 27.51}}

In [297]:
for dist, payload in city_district_temp.items():
    city_district_temp_final[dist] = round(payload['temp']/ payload['cnt'], 2)
city_district_temp_final

{'центральный  фо': 3.7,
 'северо-западный фо': 0.37,
 'уральский фо': -0.35,
 'приволжский фо': 1.67,
 'сибирский фо': -4.27,
 'дальневосточный фо': -6.61,
 'южный фо': 6.34,
 'северо-кавказский фо': 6.88}

### С помощью подхода MapReduce посчитайте  
#### 3. среднюю погоду по каждому городу за представленный период;

Аналогично, чтобы сэкономить время, пропускаем map, shuffle

In [299]:
c_temp_diff = dict()

In [309]:
for city, temp in c_temp_reduced_final.items():
    for district, cities in city_district.items():
        if city in cities:
            c_temp_diff[city] = round(abs(city_district_temp_final[district] - temp),4)

In [311]:
c_temp_diff

{'оренбург': 1.25,
 'липецк': 0.6,
 'челябинск': 1.19,
 'томск': 0.56,
 'ханты-мансийск': 2.64,
 'курган': 0.65,
 'казань': 0.3,
 'киров': 1.53,
 'тула': 0.0,
 'ижевск': 1.19,
 'ставрополь': 0.03,
 'брянск': 0.88,
 'кемерово': 0.35,
 'псков': 2.13,
 'петрозаводск': 0.5,
 'йошкар-ола': 0.59,
 'улан-удэ': 0.74,
 'иркутск': 0.88,
 'калининград': 4.07,
 'мурманск': 6.13,
 'иваново': 1.76,
 'омск': 1.78,
 'волгоград': 0.84,
 'нальчик': 0.07,
 'санкт-петербург': 1.29,
 'чебоксары': 0.38,
 'орел': 0.74,
 'элиста': 0.86,
 'сыктывкар': 2.29,
 'краснодар': 2.76,
 'ростов': 4.25,
 'самара': 0.69,
 'биробиджан': 3.83,
 'вологда': 0.15,
 'анадырь': 6.02,
 'владимир': 1.2,
 'грозный': 0.83,
 'петропавловск-камчатский': 4.27,
 'нижний новгород': 0.88,
 'якутск': 10.57,
 'кызыл': 4.28,
 'кострома': 2.1,
 'тамбов': 0.72,
 'саранск': 1.22,
 'екатеринбург': 0.23,
 'рязань': 0.29,
 'хабаровск': 5.08,
 'магадан': 6.22,
 'абакан': 2.62,
 'пенза': 1.79,
 'астрахань': 1.48,
 'пермь': 1.83,
 'воронеж': 2.13,
 